In [32]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/CloudProjectMine/creditcard.csv'

df = pd.read_csv(file_path)

print(df.shape)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(284807, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [33]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

X = df.drop("Class", axis=1)
y = df["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

sm = SMOTE(random_state=42)
X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)

model = RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=42)
model.fit(X_train_smote, y_train_smote)

y_pred = model.predict(X_test)
y_scores = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_scores))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.88      0.80      0.84       148

    accuracy                           1.00     85443
   macro avg       0.94      0.90      0.92     85443
weighted avg       1.00      1.00      1.00     85443

ROC-AUC Score: 0.9493089563565559


In [34]:
from sklearn.metrics import precision_recall_curve
import numpy as np
import joblib

y_scores = model.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)
best_index = np.argmax(f1_scores)
best_threshold = thresholds[best_index]

print(f"Best Threshold: {best_threshold:.4f}")
print(f"Best F1 Score: {f1_scores[best_index]:.4f}")

joblib.dump(model, "model.pkl")
with open("threshold.txt", "w") as f:
    f.write(str(best_threshold))


Best Threshold: 0.6700
Best F1 Score: 0.8444


In [35]:
import joblib
model = joblib.load("model.pkl")

In [36]:
with open("threshold.txt", "w") as f:
    f.write(str(best_threshold))

In [37]:
#Threshold test (Normal Transaction)
import numpy as np
import joblib

model = joblib.load("model.pkl")
with open("threshold.txt", "r") as f:
    threshold = float(f.read())

sample = X_test.iloc[[0]]
probability = model.predict_proba(sample)[0][1]

if probability >= threshold:
    print(f"Fraud detected with confidence {probability * 100:.2f}%")
else:
    print(f"Normal transaction with confidence {(1 - probability) * 100:.2f}%")


Normal transaction with confidence 100.00%


In [38]:
#Threshold test (Fraud Transaction)
fraud_samples = X_test[y_test == 1]

sample = fraud_samples.iloc[[0]]
proba = model.predict_proba(sample)[0][1]

if proba >= threshold:
    print(f"Fraud detected with confidence {proba * 100:.2f}%")
else:
    print(f"Normal transaction with confidence {(1 - proba) * 100:.2f}%")


Fraud detected with confidence 98.00%


In [39]:
#Fire-base Key Upload (json)
from google.colab import files
uploaded = files.upload()

Saving cloudprojectfirebase-firebase-adminsdk-fbsvc-f4c7af3555.json to cloudprojectfirebase-firebase-adminsdk-fbsvc-f4c7af3555.json


In [40]:
!pip install firebase-admin

In [41]:
#Fire-store Connection
import firebase_admin
from firebase_admin import credentials, firestore

if not firebase_admin._apps:
    cred = credentials.Certificate("cloudprojectfirebase-firebase-adminsdk-fbsvc-f4c7af3555.json")
    firebase_admin.initialize_app(cred)

db = firestore.client()

In [42]:
import uuid
from datetime import datetime

sample = X_test.iloc[[0]]
proba = model.predict_proba(sample)[0][1]
is_fraud = proba >= threshold

record = {
    "transaction_id": str(uuid.uuid4()),
    "features": {k: float(v) for k, v in sample.to_dict(orient="records")[0].items()},
    "probability": float(proba),
    "is_fraud": bool(is_fraud),
    "timestamp": datetime.utcnow()
}

db.collection("transactions").document(record["transaction_id"]).set(record)

update_time {
  seconds: 1748272597
  nanos: 690116000
}

In [43]:
!pip install flask flask-ngrok

In [44]:
!pip install flask pyngrok --quiet

In [45]:
!ngrok config add-authtoken 2xY69qsxHWmaV4H0Yb7LrglQeu8_7ZbxZG2c2YDZTGrAP1LAP

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [46]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import joblib
import numpy as np
import uuid
from datetime import datetime
import firebase_admin
from firebase_admin import credentials, firestore
import threading

model = joblib.load("model.pkl")
with open("threshold.txt", "r") as f:
    threshold = float(f.read())

if not firebase_admin._apps:
    cred = credentials.Certificate("cloudprojectfirebase-firebase-adminsdk-fbsvc-f4c7af3555.json")
    firebase_admin.initialize_app(cred)
db = firestore.client()

app = Flask(__name__)

@app.route("/")
def index():
    return "Fraud Detection API is online"

@app.route("/predict", methods=["POST"])
def predict():
    try:
        data = request.get_json()
        features = np.array(list(data["features"].values())).reshape(1, -1)
        proba = model.predict_proba(features)[0][1]
        is_fraud = proba >= threshold

        record = {
            "transaction_id": str(uuid.uuid4()),
            "features": data["features"],
            "probability": float(proba),
            "is_fraud": bool(is_fraud),
            "timestamp": datetime.utcnow()
        }

        db.collection("transactions").document(record["transaction_id"]).set(record)

        return jsonify({
            "fraud_probability": round(proba, 4),
            "is_fraud": bool(is_fraud)
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 400

def run_app():
    app.run()

public_url = ngrok.connect(5000)
print("🚀 API is live at:", public_url)
threading.Thread(target=run_app).start()

🚀 API is live at: NgrokTunnel: "https://1cfc-34-147-69-180.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'


In [47]:
#API test (normal transaction)
import requests

url = "https://3511-34-147-69-180.ngrok-free.app/predict"

sample_features = {
    "features": {
        "Time": 127294,
        "V1": -2.5373, "V2": 1.8908, "V3": -0.8406, "V4": -1.1028,
        "V5": -2.1057, "V6": 0.3678, "V7": -2.7378, "V8": -3.5433,
        "V9": -0.4831, "V10": 0.1127, "V11": -1.6126, "V12": 0.6359,
        "V13": 0.4069, "V14": 0.8544, "V15": 0.5058, "V16": -0.9478,
        "V17": 0.5573, "V18": 1.5466, "V19": -0.7890, "V20": -1.9596,
        "V21": 4.3282, "V22": -2.1673, "V23": 1.0226, "V24": -0.2411,
        "V25": -1.0949, "V26": -0.4754, "V27": -0.1772, "V28": -0.0827,
        "Amount": 24.00
    }
}

response = requests.post(url, json=sample_features)
print(response.json())

 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [26/May/2025 15:16:47] "POST /predict HTTP/1.1" 200 -


{'fraud_probability': 0.0, 'is_fraud': False}
